In [15]:
from utility_functions import load_file, pickle_file, starting_run, finished_run
from data_reading_functions import data_dir, read_data, died_reference
import pandas as pd
import numpy as np

In [16]:
summary_table = load_file("filtered_dataset.pickle")
codes_table = load_file("filtered_dataset_codes.pickle")
index = summary_table["max_year"].astype("int")
death_records = []

In [17]:
for dataset in ["sedd", "sid", "sasd"]:
    for year in ["2016", "2017", "2018"]:
        starting_run(dataset + year)
        data_file = read_data(died_reference[dataset][year], f"MD_{dataset.upper()}_{year}_CORE.asc")
        deaths = data_file.astype({"Died": "int"}).join(index, how="inner", on="visit_link").query(f"max_year >={year} and Died == 1")
        death_records.append(deaths)
        del data_file
        del deaths

Starting sedd2016 10:50:07.053331
Starting sedd2017 10:50:36.932835
Starting sedd2018 10:51:05.076345
Starting sid2016 10:51:40.285776
Starting sid2017 10:51:49.461719
Starting sid2018 10:51:58.728745
Starting sasd2016 10:52:10.784327
Starting sasd2017 10:52:58.800131
Starting sasd2018 10:53:41.162785


In [18]:
#Dropping duplicates due to some HCUP data vagaries
deaths = pd.concat(death_records).sort_values("record_id", ascending=False).drop_duplicates('visit_link').set_index('visit_link')

In [19]:
deaths

,Died,record_id,max_year
visit_link,,,
2214480,1,242018405120635,2018
4456081,1,242018405077698,2018
2167455,1,242018404758217,2018
12443747,1,242018404694236,2018
3422623,1,242018404563721,2018
...,...,...,...
3965560,1,242017100025667,2018
3562996,1,242017100021089,2018
4436678,1,242017100017419,2018


In [20]:
invalid_deaths = deaths.join(summary_table, how="right", lsuffix="_died").query("Died == 1 and record_id == initial_record_id")
fully_filtered_summary = summary_table.join(deaths["Died"], how="left").drop(index=invalid_deaths.index)
fully_filtered_summary["Died"] = fully_filtered_summary["Died"].fillna(0)
print(f"dropped {invalid_deaths.shape[0]} invalid patients due to death on initial record.")

dropped 143 invalid patients due to death on initial record.


In [21]:
fully_filtered_summary["Died"].value_counts()

0.0    15648
1.0      368
Name: Died, dtype: int64

In [22]:
pickle_file("fully_filtered_summary.pickle", fully_filtered_summary)
pickle_file("fully_filtered_codes.pickle", codes_table.loc[fully_filtered_summary.index])